In [1]:
print('Aloitettu: Liiga_GetPlayersStatsBySeason \n')

Aloitettu


In [3]:
# Haetaan kaikki ladattavat kaudet erikseen määritetystä tiedostosta

import csv
urls = []

# Lue csv-tiedoston sarake ilman otsikkoa ja tallenna listaan urlit
with open('LiigaKaudet.csv', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    urls = list(reader)

print('Ladattavat kaudet: ')    
for url in urls:
    print(url)

print("")

['https://www.liiga.fi/fi/tilastot/2020-2021/runkosarja/pelaajat/']
['https://www.liiga.fi/fi/tilastot/2019-2020/runkosarja/pelaajat/']
['https://www.liiga.fi/fi/tilastot/2018-2019/runkosarja/pelaajat/']


In [4]:
import pandas as pd
import bs4
import requests

from bs4 import BeautifulSoup
from urllib.request import urlopen

In [5]:
# Kerätään tiedot webscrapperilla ja tallennetaan dataframeen

first_url = True

list_of_players = []

# Käydään yksi url-osoite kerrrallaan läpi
for url in urls:
    html = urlopen(url[0])
    soup = BeautifulSoup(html, 'lxml')

    # Sivun otsikko
    title = soup.find('title')
    title_splitted = title.string.split()
    season = title_splitted[5]
    print("Avattu: " + season)
    
    
    # Tarkista onko ensimmäinen kierros --> Luo dataframe
    if first_url:
        for all_data in soup.find_all('table', {'id': 'stats'}):
            
            # Haetaan otsikkorivi
            for headers in all_data.find_all('thead'):
                for header in headers.find_all('tr'):
                    column_names = []
                    column_names.append("Kausi")
                    column_names.append("Linkki")
                    for column in header.find_all('td'):
                        column_names.append(' '.join(column.text.split()))
        
        df = pd.DataFrame(columns = column_names)
        print("Luotu df: ")
        print(column_names)
    
    # Haetaan pelaajien tilastot taulukosta ja lisätään df
    for all_data in soup.find_all('table', {'id': 'stats'}):
        
        #Pelaajatilastot: kaikki pelaajat yhteen listaan
        for table_of_stats in all_data.find_all('tbody'):
            
            for stats in table_of_stats.find_all('tr'):
                player = []
                player.append(season)
                
                link = ""
                for a in stats.find_all('a', href=True):
                    link = a.get('href')
                player.append(link)
                
                for detail in stats.find_all('td'):
                    player.append(' '.join(detail.text.split()))
                list_of_players.append(player)

                # Tallennetaan pelaajatiedot dataframeen
                df.loc[len(df)] = player
    
    print("Tallennettu kausi " + season + " dataframeen \n")
    first_url = False            
    

Avattu: 2020-2021
Luotu df: 
['Kausi', 'Linkki', '#', 'Nimi', 'Joukkue', 'PP', 'O', 'M', 'S', 'P', 'R', '+', '-', '±', 'YV', 'AV', 'VM', 'L', 'L%', 'A', 'A%', 'Aika']
Tallennettu kausi 2020-2021 dataframeen 

Avattu: 2019-2020
Tallennettu kausi 2019-2020 dataframeen 

Avattu: 2018-2019
Tallennettu kausi 2018-2019 dataframeen 



In [ ]:
# Tarkista dataframen tiedot
print('DataFrame: \n')
print(df)
print("")

In [ ]:
print("Tietotyypit: \n")
print(df.dtypes)
print("")

In [ ]:
print("Siivotaan datasettiä; sarakkeiden nimiä, tietotyyppejä ja arvoja \n")

# Muutetaan sarakkeiden nimet
df.rename(columns= {'#': 'Pistepörssi_sijoitus', 'PP' : 'Pelipaikka', '+': 'Plussat', '-': 'Miinukset', '±': 'PlusMiinus', 'Aika': 'Peliaika'}, inplace=True)

# Korjataan arvoja
df['Plussat'] = df['Plussat'].replace(['-'], None)
df['Miinukset'] = df['Miinukset'].replace(['-'], None)
df['PlusMiinus'] = df['PlusMiinus'].replace(['-'], None)
#df['L%'] = df['L%'].replace([','], '.')
#df['A%'] = df['A%'].replace([','], '.')

# Muokataan tietotyyppejä
df['Joukkue'] = df['Joukkue'].astype('category')
df['Pelipaikka'] = df['Pelipaikka'].astype('category')
df['O'] = df['O'].astype('int64')
df['M'] = df['M'].astype('int64')
df['S'] = df['S'].astype('int64')
df['P'] = df['P'].astype('int64')
df['R'] = df['R'].astype('int64')
df['Plussat'] = df['Plussat'].astype('int64')
df['Miinukset'] = df['Miinukset'].astype('int64')
df['PlusMiinus'] = df['PlusMiinus'].astype('int64')
df['YV'] = df['YV'].astype('int64')
df['AV'] = df['AV'].astype('int64')
df['VM'] = df['VM'].astype('int64')
df['L'] = df['L'].astype('int64')
#df['L%'] = df['L%'].astype('int64')
df['A'] = df['A'].astype('int64')
#df['A%'] = df['A%'].astype('int64')

In [ ]:
# Muokataan datasetin tietueita käytettävämpään muotoon 

"""
Korjauksia: L%, A%
Päättelyä: kausi, nimi, plussat, miinukset, plusmiinus, L%, A%, peliaika
"""

#Kausi
df['Kausi_jaettu'] = df['Kausi'].str.split('-')
df['Kausi_aloitus'] = df.apply(lambda row: row.Kausi_jaettu[0], axis=1)
df['Kausi_lopetus'] = df.apply(lambda row: row.Kausi_jaettu[1], axis=1)
df.drop('Kausi_jaettu', inplace=True, axis=1)

#Nimi
df['Tulokas'] = df['Nimi'].str.contains('*', regex=False)
df['Ulkomaille'] = df['Nimi'].str.contains('#', regex=False)
df['Nimi'] = df['Nimi'].map(lambda cell: cell.lstrip('#').rstrip('aAbBcC'))
df['Nimi'] = df['Nimi'].map(lambda cell: cell.lstrip('*').rstrip('aAbBcC'))

#Nimi
df['Nimi_jaettu'] = df['Nimi'].str.split(',')
df['Etunimi'] = df.apply(lambda row: row.Nimi_jaettu[1], axis=1)
df['Sukunimi'] = df.apply(lambda row: row.Nimi_jaettu[0], axis=1)
df['Koko_nimi'] = df['Sukunimi'] + ' ' + df['Etunimi']
df.drop('Nimi_jaettu', inplace=True, axis=1)
#df.drop('Nimi', inplace=True, axis=1)

#Peliaika
df['Peliaika_jaettu'] = df['Peliaika'].str.split(':')
df['Peliaika_minuutit'] = df.apply(lambda row: row.Peliaika_jaettu[0], axis=1)
#df['Peliaika_sekuntit'] = df.apply(lambda row: row.Peliaika_jaettu[1], axis=1)
df.drop('Peliaika_jaettu', inplace=True, axis=1)

#pelaajaid
df['Linkki_temp'] = df['Linkki'].str.split('/')
df['pelaaja_id'] = df.apply(lambda row: row.Linkki_temp[3], axis=1)
df.drop('Linkki_temp', inplace=True, axis=1)

In [ ]:
print("Uudet tietotyypit: \n")
print(df.dtypes)
print("")

In [9]:
# Tallennetaan datasetti csv-tiedostoon (tarvittaessa vanhan korvaaminen)

df.to_csv(r'C:\Users\35840\Documents\JohdantoDatatieteisiin2021\JohdantoDatatieteisiin2021\Harjoitustyö\PlayersStats_by_Season.csv', index = False, header=True)

print("Tallennettu PlayersStats_by_Season.csv \n")

In [ ]:
print("Suoritettu Liiga_GetPlayersStatsBySeason")